# Dogs Vs Cats com Torch

Fiz este notebook para aprender PyTorch sendo que eu estou a mudar de Tensorflow para PyTorch.

# Imports

* `zipfile` - Extrair as imagens de treino e testes dos ficheiros ZIP
* `time` - Guardar o timestamp quando alguma métrica é guardada em disco
* `os` - Útil para navegar nos ficheiros
* `random` - Selecionar um número aleatório para a seed
* `numpy` - Para baralhar e para matemática em geral
* `pandas` - Criar o ficheiro de submissão .csv
* `shutil` - Mover imagens
* `PIL.Image` - Carregar imagens de validção
* `collections` - Criar dicionários aninhados
* `tqdm` - Barras de progresso
* `torch.utils.data.DataLoader` - Criar conjuntos(batches) de forma fácil
* `torch.utils.data.Dataset` - Criar um dataset personalizado com os dados de validação
* `torch.utils.data.sampler.SubsetRandomSampler` - Escolher amostras de um sub conjunto de índices
* `torchvision.datasets` - Carregar imagens e labels de um diretório raiz
* `torchvision.transforms` - Aplicar transformações de uma dado dataset
* `torchvision.models` - Carregar modelos pretreinados
* `torchvision.utils.makegrid` - Plotar múltiplas imagens
* `torch` - Métodos comuns do PyTorch
* `torch.nn` - Módulos para construir layers
* `torch.nn.functional` - Métodos como funções de ativação
* `torch.optim` - Otimizadores
* `matplotlib.pyplot` - Plotar
* `matplotlib.style` - Trocar o estilo dos plots

In [ ]:
import zipfile
import time
import os
import random
import numpy as np
import pandas as pd
import shutil
from PIL import Image
import collections
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms, models
from torchvision.utils import make_grid
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from matplotlib import style
%matplotlib inline

Se nós corremos o kernel de novo sem reiniciá-lo, o `metrics.log` mantém-se no disco e vai acumular métricas da última vez que corremos o notebook.

Para prevenir isso vamos removê-lo.

In [ ]:
%rm '/kaggle/working/metrics.log'

Escolher o dispositivo que vai correr o modelo tanto durante o treino como durante a validação e testes

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# DataLoader

Descomprimir os conjuntos de treino e test que estão no diretório `/kaggle/working/`

In [ ]:
# Extrair os ZIPs
with zipfile.ZipFile("../input/dogs-vs-cats/train.zip", "r") as unzip:
    unzip.extractall(".")

    
with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip", "r") as unzip:
    unzip.extractall(".")

Criar outro diretório com 2 pastas dentro (uma para cada classe/label). Se elas já existirem não criaremos novamente.

In [ ]:
os.makedirs("my_train/dogs", exist_ok=True)
os.makedirs("my_train/cats", exist_ok=True)

Aqui as imagens de treino são movida para a pasta da classe correta dentro de `my_train`.

In [ ]:
root = "train"
imgs = os.walk(root).__next__()[2]
folders = {
    "cat": "my_train/cats",
    "dog": "my_train/dogs"
}

for img in tqdm(imgs):
    label = img.split(".")[0]
    
    old_path = os.path.join(root, img)
    new_path = os.path.join(folders[label], img)
    
    shutil.move(old_path, new_path)
    
    

Aqui as imagens que estão dentro de `my_train` são transformadas e divididas em 2 datasets: `train` and `val` - um para treino e outro para validação.

Provavelmente estão a perguntar - "Porque colocaste esse valores específicos no método Normalize?" - Quando usamos datasets pré-treinados no ImageNet, estes valores foram estimado por eles (ImageNet) utilizando milhões de imagens. Logo isso dá uma boa estimativa em termo de média e desvio-padrão.

"E porquê 2 arrays com 3 valores cada?" - O primeiro array é para média e o segundo é para o desvio-padrão. E usamos 3 valores, porque as imagens são normalizadas na dimensão de profundidade (na dimensão dos canais) e nós temos imagens em RGB, logo temos 3 canais (um valor para cada canal).

In [ ]:
# Tamanho da imagem
IMG_SIZE = 260

# Transformações
data_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomHorizontalFlip(0.5)
])

# Carregar as imagens e labels
dataset = datasets.ImageFolder(root="my_train", transform=data_transforms)

# Obter o nome de cada label numérica
classes = dataset.classes

# 20% do dataset de treino será usado para
# validação
val_split = 0.2
dataset_size = len(dataset)
idxs = list(range(dataset_size))

# Número de imagens de validação
split = int(val_split * dataset_size)

# Gerar um seed aleatório e baralhar os índices
np.random.seed(random.randint(0, 99999))
np.random.shuffle(idxs)

dataset_size = {
    "train": dataset_size - split,
    "val": split
}

# Fazer com que os data sampler selecionem N imagens únicas
# de cada dataset
train_sampler = SubsetRandomSampler(idxs[:-split])
val_sampler = SubsetRandomSampler(idxs[-split:])

Dividir o dataset em 2 dataloaders e cada um vai receber um sub conjunto de imagens únicas, divididas em batches (conjuntos).

In [ ]:
dataloaders = {
    "train": DataLoader(dataset, batch_size=64, sampler=train_sampler),
    "val": DataLoader(dataset, batch_size=64, sampler=val_sampler),
}

Apenas uma simples função para mostra 4 imagens do dataloader de treino.

Algumas pessoas não entendem o porquê destas linhas de código:
`mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
inp = std * inp + mean
inp = np.clip(inp, 0, 1)`

Se está a ler desde o início deverá saber que usamos a média e o desvio-padrão para normalizar as imagens, logo as 2 primeiras linhas estão explicadas. Agora o porquê de `inp` ser  igual a `std * inp + mean`.

Em estatística temos algo chamado de ***Standard Scores*** ou ***Z-score*** e nós podemos usá-lo para normalizar valores (apenas se soubermos a média e o desvio-padrão da população). A expressão é:
$\frac{X - \mu}{\sigma}$, onde $X$ é a nossa imagem, $\mu$ a média e $\sigma$ o desvio-padrão.

Mais informações acerca do deste assunto, [aqui](https://en.wikipedia.org/wiki/Standard_score)

Então as nossas imagens foram normalizadas com essa expressão e devido a isso o matplotlib não plotará as imagens como esperamos, pois os valores dos pixeis estão normalizados. Para "desnormalizar" precisamos de reverter a expressão do ***z-score***:
$X * \sigma + \mu$

Como podemos ver todas as operações da expressão foram invertidas e isso reflete-se no seguinte código:
`std * inp + mean`

Agora, o que é o `clip()` faz? *Clipar* é pegar pegar em todos os valores e traduzi-los (passá-los) para dentro de um certo intervalo que neste caso é um intervalo = [0,1]. Se durante a "desnormalização" alguns valores forem < 0 ou > 1, eles serão transformados em valores próximos de 0 (se o valor for < 0) ou em valores próximos de 1 (se o valor for > 1)  

In [ ]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1,2,0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.grid(False)
    plt.imshow(inp)
    
    if title != None:
        plt.title(title)
    
    plt.pause(0.001)

    
features, labels = next(iter(dataloaders["train"]))
features = features[:4]
labels = labels[:4]
out = make_grid(features)

imshow(out, title=[classes[x] for x in labels])

# DataLoader para testes
Este dataset personalizado pega em todos os nomes dos ficheiros no diretório `test1` e quando iteramos em todos esses ficheiros, o dataset vai ler cada uma das imagens, aplicar transformações (caso as passemos) e retornar a image e o nome dela (que será útil para a criaćão do CSV para submissão).

In [ ]:
class TestDataset(Dataset):
    
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.paths = os.walk(root_dir).__next__()[2]
        self.transform = transform
    
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_path = os.path.join(self.root_dir, self.paths[idx])
        
        img = Image.open(img_path)
        
        if self.transform != None:
            img = self.transform(img)
        
        return img, self.paths[idx].split(".")[0]

Criar um dataloarder com o dataset personalizado. As transformações são as mesmas exceto o `RandomFlip` que é excluido, porque as imagens não devem ser transformadas neste dataloader de testes.

In [ ]:
test_dataset = TestDataset(root_dir="./test1", transform=transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]))

test_dataloader = DataLoader(test_dataset, batch_size=64)

In [ ]:
imgs, _ = next(iter(test_dataloader))
imgs = imgs[:4]
out = make_grid(imgs)
imshow(out)

# Função de Treino

A função mais importante, onde o treino e a validação acontecem.

In [ ]:
def train(model, loss_fn, optimizer, num_epochs=5, model_name="model", lr_scheduler=None):
    
    for epoch in range(num_epochs):
        print("Epoch {}/{}\n".format(epoch+1, num_epochs))
        
        for step in dataloaders:
            
            # Todas as layers com trainning=True
            if step == "train":
                model.train()
            
            # Todas as layers com trainning=False
            else:
                model.eval()
            
            # Custo
            l = 0
            # Acurácia
            acc = 0
            
            for X_batch, y_batch in tqdm(dataloaders[step]):
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                
                # Zerar o gradiente, porque cada batch deve ter o seu próprio gradiente
                optimizer.zero_grad()
                
                
                # Treinar
                # Comg gradientes =True em todas as layers
                with torch.set_grad_enabled(step == "train"):
                    preds = model(X_batch)
                    
                    # o max retorna uma tupla com os valores máximos e os respetivos 
                    # índices. Eu quero apenas os índices, pois o CrossEntropy
                    # retorna um Tensor com todas as probabilidades "desnormalizadas"
                    # para cada label. O índice da probabilidade mais alta é o label
                    # predito
                    _, max_idxs = torch.max(preds, dim=1)
                    
                    
                    # Calcular erros
                    loss = loss_fn(preds, y_batch)
                    
                    # Treino a sério abaixo
                    if step == "train":
                        ## regularizador L2 
                        #l2_factor = 0.005
                        #l2_reg = l2_factor * np.sum([(w**2).sum() for w in model.parameters()])
                        #loss = loss + l2_reg
                        
                        # Backprop + atualização do pesos
                        loss.backward()
                        optimizer.step()
                    
                # Métricas
                l += loss.item() * X_batch.size(0)
                acc += torch.sum(max_idxs == y_batch.data)
                
                # Escrever as métricas referentes ao batch atual, em um ficheiro
                with open(model_name+".log", "a") as f:
                    f.write("{},{},{},{}\n".format(
                        round(time.time(), 3),
                        step,
                        torch.sum(max_idxs == y_batch.data).item() / y_batch.size(0),
                        loss.item()
                    ))
                
            # Calcular a média das métricas durante cada epoch
            epoch_loss = l / dataset_size[step]
            epoch_acc = acc.double() / dataset_size[step]
            
            # Atualizar learning rate
            # Podemos alterar o learning rate a cada epoch 
            # utilizando o learning rate scheduler
            if step == "train" and lr_scheduler != None:
                lr_scheduler.step()
            
            print("{} Acc: {:.3f} - Loss: {:.3f}".format(step, epoch_acc, epoch_loss))
        print()
    
    return model

# Modelo personalizado
Aqui é a minha célula de testes, onde eu tento criar alguns modelos personalizados, mas eles não funcionaram até agora.

**Podem passar para a próxima célula, já que este modelo não será usado**

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 64, 5, stride=(2,2))
        self.res_conv1 = None
        self.conv2 = nn.Conv2d(64, 64, 3, padding=3//2)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=3//2)
        self.res_conv3 = None
        self.conv4 = nn.Conv2d(64, 64, 3, padding=3//2)
        self.conv5 = nn.Conv2d(64, 64, 3, padding=3//2)
        
         
        
        x = torch.randn(3, IMG_SIZE, IMG_SIZE).view(-1, 3, IMG_SIZE, IMG_SIZE)
        self.get_flatten = None
        
        # Obter o número de parâmetros na última layer de convolução
        self.forward_convolutions(x)
        
        self.dropout = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(self.get_flatten, 2)
    
    def forward_convolutions(self, X):
        X = F.relu( self.conv1(X) )
        self.res_conv1 = X
        
        X = F.relu( self.conv2(X) )
        X = F.relu( self.conv3(X) )
        X = self.res_conv1 + X
        self.res_conv3 = X
        
        X = F.relu( self.conv4(X) )
        X = F.relu( self.conv5(X) )
        X = self.res_conv3 + X
        
        
        # Se estivermos a tentar obter o número de
        # parâmetros na última convolução
        if self.get_flatten == None:
            self.get_flatten = 1

            for sz in X.size()[1:]:
                self.get_flatten *= sz
        
        
        return X
    
    def forward(self, X):
        X = self.forward_convolutions(X)
        
        X = X.view(-1, self.get_flatten)
        X = self.dropout(X)
        
        return self.fc1(X)

    
    def calc_outputs(self):
        # Calcular o tamanho do output
        # de cada layer de convolução
        
        conv_idx = 1
        
        out_h_prev = out_w_prev = IMG_SIZE
        
        for layer in self.children():
            if isinstance(layer, nn.Conv2d):
                inp = (out_h_prev, out_w_prev)
                out = layer.out_channels
                k = layer.kernel_size
                s = layer.stride
                p = layer.padding

                out_h = ( ( inp[0] + (2*p[0]) - k[0] ) // s[0] ) + 1
                out_w = ( ( inp[1] + (2*p[1]) - k[1] ) // s[1] ) + 1 
                
                print("Output Convolution Layer {} = ({},{})".format(
                    conv_idx,
                    out_h//2, # convoluções são divididas por 2, pois todas elas passam por uma pool layer 
                    out_w//2 # com uma pool window de tamanho 2x2 e com stride 2x2, o que faz ficar com metade do tamanho
                ))
                
                out_h_prev, out_w_prev = out_h, out_w
                
                conv_idx += 1

**Apenas para testar o output final do nosso feature extractor**

In [ ]:
#test = CNN()

#test.calc_outputs()


# Train

Aqui eu testo diferentes parâmetros para o meu próprio modelo e para o modelo resnet18 pré-treinado.

Eu verifiquei que:
* Dropout com 50% de probs. de os neurónios serem temporariamente "mortos", diminui o custo de validação
* Otimizador Adam a começar com um learning rate de 0.001 converge bastante rápido
* O learning rate step com um gamma = 5 e apenas mudar a cada 5 epochs, ajuda a aumentar a acurácia e a diminuir o custo do conjunto de validação

In [ ]:
## Depois de akguns testes verifiquei que 10 epochs são suficientes para o resnet18
model_names = {
    #"model_cnn_3conv,3k,1epoch": 1,
    #"model_cnn_3conv,3k,3epoch": 3,
    #"model_resnet18,5epoch": 5,
    "model_resnet18,10epoch": 10,
}

for k,v in model_names.items():
    ## Transfer Learning
    model = models.resnet18(pretrained=True)
    n_features = model.fc.in_features
    
    # Congelar as layers, exceto a minha nova layer Linear/Dense
    for layer in model.parameters():
        layer.requires_grad = False
    
    model.fc = nn.Sequential(collections.OrderedDict([
        ("fc_dropout1", nn.Dropout(0.5)),
        ("fc_softmax", nn.Linear(n_features, 2))
    ]))
    model = model.to(device)
    
    
    ## O meu modelo
    #model = CNN().to(device)
    
    # Função custo
    loss_fn = nn.CrossEntropyLoss()

    # Otimizador
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    ## Decadência do learning rate por um fator de 5 a cada 5 epochs
    exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=5)
    model = train(model, loss_fn, optimizer, num_epochs=v, 
                  model_name=k, lr_scheduler=exp_lr_scheduler)
    
    
    ## Quando temos múltiplos model_names é bom eliminar
    ## o objeto do modelo, para prevernir o uso de pesos treinados
    ## do modelo anterior
    #del model

# Visualizar métricas

Aqui criamos um grande dicionário para guardar todas as acurácias de treino e teste e custos (aqueles que guardamos em um ficheiro durante o treino/validação)

In [ ]:
nested_dict = lambda: collections.defaultdict(nested_dict)

models_metrics = nested_dict()

for k,v in model_names.items():

    train_accs = []
    train_losses =[]
    val_accs = []
    val_losses = []
    losses_removed = 0
    with open(k+".log") as f:
        for line in f:
            acc = line.split(",")[2]
            loss = line.split(",")[3]
            
            # The model goes really bad with some batches
            # To keep the plot with values between 0 and 1
            # I delete all the losses greater than 1
            if float(loss) > 1.0:
                losses_removed += 1
                continue
                

            if "train" in line:
                train_accs.append(float(acc))
                train_losses.append(float(loss))
            else:
                val_accs.append(float(acc))
                val_losses.append(float(loss))
    
    models_metrics[k]["train_accs"] = train_accs
    models_metrics[k]["train_losses"] = train_losses
    models_metrics[k]["val_accs"] = val_accs
    models_metrics[k]["val_losses"] = val_losses

## Funções de suavização

As métricas dos batches podem ser muito instáveis, logo eu tentei implementar 2 métodos para suaviazar as linhas plotadas.
A **Hanning Window** é uma função usada no processamento de sinais para suavizar valores e é definida como:
$\frac{1}{2} - \frac{1}{2}cos(\frac{2\pi n}{M-1})$, onsde $n$ é o valor atual e $M$ é o número total de valores.

Se multiplicarmos o resultado pelo valor atual, ele deve ser suavizado (numéricamente ele fica mais pequeno, mas visualmente a diferença é quase imperceptível).

A **Moving Average** é basicamente a média que usa todos os valores calculados previamente. Então quando chegamos ao último valor, o primeiro valor não tem quase impacto na média. Eu tentei implementar a versão usada no Tensorboard, mas não funcionou (provavelmente eu preciso colocar os devidos expoentes para cada valor, para assim os valores mais próximos do atual terem mais impacto que os mais antigos). Acabei apenas por usar a hanning window.

In [ ]:
def hanning_window(metric):
    for i in range(len(metric)):
        n = metric[i]
        h_metric = 0.5 - 0.5 * np.cos( (2*np.pi*n) / (len(metric)-1) )
        metric[i] *= n

    return metric


def moving_average(metric, w):
    
    last_smooth = metric[0]
    smoothed_metric = []
    
    for i, p in enumerate(metric):
        if i == 0:
            smooth = p
        else:
            smooth = last_smooth * w + (1 - w) + p 
        
        smoothed_metric.append( smooth )
        last_smooth = smooth
        
    return smoothed_metric

Calcular cada métrica e transformar as métricas guardar utilizando a Hanning Window

In [ ]:
for k,v in model_names.items():
    
    mean_train_acc, mean_train_loss = np.mean(models_metrics[k]["train_accs"]), np.mean(models_metrics[k]["train_losses"])
    mean_val_acc, mean_val_loss = np.mean(models_metrics[k]["val_accs"]), np.mean(models_metrics[k]["val_losses"])
    
    models_metrics[k]["mean_train_acc"] = mean_train_acc
    models_metrics[k]["mean_train_loss"] = mean_train_loss
    models_metrics[k]["mean_val_acc"] = mean_val_acc
    models_metrics[k]["mean_val_loss"] = mean_val_loss

    
        
    models_metrics[k]["train_accs"] = hanning_window(models_metrics[k]["train_accs"])
    models_metrics[k]["train_losses"] = hanning_window(models_metrics[k]["train_losses"])
    models_metrics[k]["val_accs"] = hanning_window(models_metrics[k]["val_accs"])
    models_metrics[k]["val_losses"] = hanning_window(models_metrics[k]["val_losses"])

Agora posso usar o dicionário grande para plotar a acurácina e o custo de treino assim como as de validação.

In [ ]:
style.use("ggplot")

for k,v in model_names.items():
    f, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6))

    ax1.set_title(k)
    
    ax1.plot(np.arange(len(models_metrics[k]["train_accs"])), models_metrics[k]["train_accs"], label="Acc - Mean: {:.2f}".format(models_metrics[k]["mean_train_acc"]))
    ax1.plot(np.arange(len(models_metrics[k]["train_losses"])), models_metrics[k]["train_losses"], label="Loss - Mean: {:.2f}".format(models_metrics[k]["mean_train_loss"]))

    ax1.legend()
    
    ax2.plot(np.arange(len(models_metrics[k]["val_accs"])), models_metrics[k]["val_accs"], label="Val Acc - Mean: {:.2f}".format(models_metrics[k]["mean_val_acc"]))
    ax2.plot(np.arange(len(models_metrics[k]["val_losses"])), models_metrics[k]["val_losses"], label="Val Loss - Mean: {:.2f}".format(models_metrics[k]["mean_val_loss"]))
    ax2.legend()
    
    # Guardar o plot como uma imagem para poder analisar e comparar
    # com outros testes que fiz
    plt.savefig(k+".png")

# Testar

Aqui eu apenas usei o dataloader de test para prever o label/classe de cada imagem. Coloquei o nome da imagem como key e o label predito como valor no dicionário.

In [ ]:

pred_dict = dict()

with torch.no_grad():
    for X_batch,names in tqdm(test_dataloader):
        X_batch = X_batch.to(device)
        
        preds = model(X_batch)
        
        _, ys = torch.max(preds, dim=1)
        
        for y, name in zip(ys, names):
            pred_dict[int(name)] = y.item()
    

Criei um DataFrame com o dicionário e com a estrutura de submissão 

In [ ]:
df_test = pd.DataFrame(pred_dict.items(), columns=["id", "label"])

Guardar o DataFrane com um ficheiro .csv

In [ ]:
df_test.to_csv("submission.csv", index=False)